In [1]:
import pandas as pd

# Load recipe data from CSV
recipes_df = pd.read_csv('../Resource/completed_recipes.csv')
print(f"Loaded {len(recipes_df)} recipes.")


Loaded 522517 recipes.


In [2]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "D*d4-0+Kl+lxfbbzh5ut"),
    ca_certs="~/http_ca.crt"
)

if es.ping():
    print("Elasticsearch connect for now!!")
else:
    print("failed to connect")

Elasticsearch connect for now!!


In [3]:
import json
import numpy as np
from elasticsearch.helpers import bulk
import re

# Define index name and sample size for development
index_name = "recipes"
sample_size = 1000  # Set the sample size for testing (adjust as needed)

# Delete the index if it already exists
es.indices.delete(index=index_name, ignore=[400, 404])

mapping = {
    "settings": {
        "analysis": {
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 2,  # Minimum length of n-grams
                    "max_gram": 3,  # Maximum length of n-grams
                    "token_chars": ["letter", "digit"]
                }
            },
            "analyzer": {
                "default": {
                    "type": "english"
                },
                "ngram_analyzer": {  # Custom N-gram analyzer for partial word search
                    "type": "custom",
                    "tokenizer": "ngram_tokenizer",
                    "filter": ["lowercase"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "recipe_id": {"type": "keyword"},
            "name": { 
                "type": "text", 
                "analyzer": "english",
                "fields": { 
                    "ngram": {  # N-gram variant for better partial matches
                        "type": "text", 
                        "analyzer": "ngram_analyzer"
                    }
                }
            },
            "description": {"type": "text", "analyzer": "english"},
            "instructions": {"type": "text", "analyzer": "english"},
            "text": {"type": "text", "analyzer": "english"},
            "calories": {"type": "float"},
            "rating": {"type": "float"},
            "image_url": {"type": "keyword"}
        }
    }
}

es.indices.create(index=index_name, body=mapping)
print(f"Created index: {index_name}")

recipes_sample = recipes_df.head(sample_size)

def clean_text(text):
    """Remove unwanted characters like c("..."), quotes, and escape sequences."""
    if not isinstance(text, str):  
        return ""  

    text = re.sub(r'c\("', '', text)
    text = re.sub(r'"\)', '', text)

    text = text.replace('\\"', '')  
    text = text.replace('"', '')    
    text = text.replace("\\", '')  
    cleaned_urls = re.sub(r'\s+', ' ', text.strip())
    # Split the string by ', ' (comma followed by a space)
    urls = cleaned_urls.split(', ')
    return urls

def clean_instructions_combined_v2(instructions):
    """Cleans and formats recipe instructions into a single, well-structured sentence."""
    if isinstance(instructions, list):
        instructions = " ".join(instructions)  # Join list into a single string
    
    if not isinstance(instructions, str):
        return ""

    # Remove unwanted wrapping (c(...)) and extra quotes
    instructions = re.sub(r'c\s*\(\s*', '', instructions)  # Remove "c("
    instructions = re.sub(r'\s*\)$', '', instructions)  # Remove trailing ")"
    instructions = instructions.strip('"')

    # Fix letter-by-letter spacing issues (e.g., "M i x" → "Mix in")
    words = instructions.split()  # Split into words
    cleaned_words = []
    buffer = ""

    for word in words:
        if len(word) == 1:  # If it's a single letter, buffer it
            buffer += word
        else:
            if buffer:
                cleaned_words.append(buffer + word)  # Merge buffer with current word
                buffer = ""
            else:
                cleaned_words.append(word)

    if buffer:  # Append remaining buffered text
        cleaned_words.append(buffer)

    instructions = " ".join(cleaned_words)

    # Remove redundant punctuation and ensure proper spacing
    instructions = re.sub(r'\.\s*\.', '.', instructions)  # Remove repeated periods
    instructions = re.sub(r'\s*\.\s*', '. ', instructions)  # Ensure proper space after periods
    instructions = re.sub(r'\s*,', ',', instructions)  # Remove spaces before commas

    # Remove leading/trailing unwanted characters (like extra quotes)
    instructions = re.sub(r'(^\"|\"$)', '', instructions)  # Remove leading/trailing quotes

    # Ensure proper formatting of sentences and remove unnecessary escape sequences
    instructions = instructions.replace('\\"', '')  # Remove escaped quotes
    instructions = instructions.replace('", "', ', ')  # Convert improperly formatted list items into a natural sentence

    # Convert sentence breaks into commas for a continuous explanation
    instructions = re.sub(r'\s*\.\s*', ', ', instructions)  # Convert periods into commas for a smoother flow

    # Fix double commas and extra spaces
    instructions = re.sub(r',\s*,+', ', ', instructions)  # Remove repeated commas
    instructions = re.sub(r'\s+', ' ', instructions)  # Remove extra spaces

    # Ensure only one period at the very end
    instructions = instructions.strip().rstrip(',') + "."  # Remove trailing comma and add a period

    return instructions



def generate_docs(df):
    for idx, row in df.iterrows():
        recipe_id = str(int(float(row.get('RecipeId', idx))))
        name = row.get('Name', '')
        description = row.get('Description', '')  # Replacing "ingredients"
        instructions = clean_instructions_combined_v2(row.get('RecipeInstructions', []))  # ✅ Clean instructions at indexing
        text = clean_text(row.get('text', ''))  # Clean text
        calories = float(row.get('Calories', 0))
        image_url = row.get('image_link', [])

        try:
            rating = float(row['AggregatedRating']) if not np.isnan(row['AggregatedRating']) else 0
        except (KeyError, TypeError, ValueError):
            rating = 0
        
        doc = {
            "_op_type": "index",
            "_index": index_name,
            "_id": recipe_id,
            "_source": {
                "recipe_id": recipe_id,
                "name": name,
                "description": description,  # Updated field
                "instructions": instructions,  # ✅ Cleaned instructions
                "text": text,  # Updated field
                "calories": calories,
                "rating": rating,
                "image_url": image_url
            }
        }
        if not np.isnan(row.get('AggregatedRating', np.nan)):
            doc["_source"]["rating"] = float(row["AggregatedRating"])
        yield doc


bulk(es, generate_docs(recipes_sample))

print(f"Indexed {len(recipes_sample)} recipes into Elasticsearch.")


C:\Users\Ned\AppData\Local\Temp\ipykernel_39372\3960969787.py:11: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])


Created index: recipes
Indexed 1000 recipes into Elasticsearch.


### Flask

In [4]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pymysql
from werkzeug.security import generate_password_hash, check_password_hash
import random
import uuid

In [18]:
app = Flask(__name__)
app.secret_key = "SOME_RANDOM_SECRET_KEY"
CORS(app)

connection = pymysql.connect(
    host='127.0.0.1',
    port=3309,
    user='root',
    password='root_password',
    db='my_database',
    cursorclass=pymysql.cursors.DictCursor
)

INDEX_NAME = "recipes"

@app.route('/search', methods=['GET'])
def search():
    query = request.args.get('q', '').strip()
    page = int(request.args.get('page', 1))
    results_per_page = 10  # Keep this aligned with frontend

    if not query:
        return jsonify({"error": "Query parameter 'q' is required"}), 400

    es_query = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["name^3", "description^2", "text", "name.ngram^2"],
                "fuzziness": "AUTO"
            }
        },
        "size": results_per_page,
        "from": (page - 1) * results_per_page
    }

    try:
        response = es.search(index=INDEX_NAME, body=es_query)
        total_count = response["hits"]["total"]["value"]  # ✅ Preserve total count
        hits = response.get("hits", {}).get("hits", [])

        if not hits:
            return jsonify({"error": f"No recipes found for '{query}'"}), 404

        results = [
            {
                "recipe_id": hit["_source"].get("recipe_id"),
                "name": hit["_source"].get("name"),
                "description": hit["_source"].get("description"),
                "instructions": hit["_source"].get("instructions"),
                "image_url": clean_text(hit["_source"].get("image_url")),  # ✅ Kept as you requested
                "calories": hit["_source"].get("calories"),
                "rating": hit["_source"].get("rating", 0),
                "score": hit["_score"]
            }
            for hit in hits
        ]

        return jsonify({"results": results, "total_count": total_count})  # ✅ Return total count

    except Exception as e:
        return jsonify({"error": f"Elasticsearch error: {str(e)}"}), 500



@app.route('/recipe/<recipe_id>', methods=['GET'])
def get_recipe(recipe_id):
    response = es.get(index=INDEX_NAME, id=recipe_id, ignore=[404])

    if not response or not response.get("found"):
        return jsonify({"error": f"Recipe ID {recipe_id} not found in Elasticsearch"}), 404

    source = response["_source"]
    source["instructions"] = clean_instructions_combined_v2(source.get("instructions", ""))
    source["image_url"] = clean_text(source.get("image_url", ""))[0] if source.get("image_url") else ""

    return jsonify(source)

@app.route('/register', methods=['POST'])
def register():
    data = request.json
    username = data.get('username')
    password = data.get('password')

    if not username or not password:
        return jsonify({"error": "Missing username or password"}), 400

    password_hash = generate_password_hash(password)

    with connection.cursor() as cursor:
        cursor.execute("SELECT user_id FROM users WHERE username = %s", (username,))
        if cursor.fetchone():
            return jsonify({"error": "Username already taken"}), 400

        cursor.execute("INSERT INTO users (username, password_hash) VALUES (%s, %s)", (username, password_hash))
        connection.commit()

    return jsonify({"message": "Registration successful"}), 200

@app.route('/login', methods=['POST'])
def login():
    data = request.json
    username = data.get('username')
    password = data.get('password')

    if not username or not password:
        return jsonify({"error": "Missing username or password"}), 400

    with connection.cursor() as cursor:
        cursor.execute("SELECT user_id, password_hash FROM users WHERE username = %s", (username,))
        user = cursor.fetchone()
        if not user or not check_password_hash(user['password_hash'], password):
            return jsonify({"error": "Invalid username or password"}), 401

        token = str(uuid.uuid4())
        cursor.execute("REPLACE INTO sessions (token, username, user_id) VALUES (%s, %s, %s)",
                       (token, username, user['user_id']))
        connection.commit()

        return jsonify({"message": "Login successful", "username": username, "token": token}), 200

@app.route('/logout', methods=['POST'])
def logout():
    auth_header = request.headers.get('Authorization')

    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]

    with connection.cursor() as cursor:
        cursor.execute("DELETE FROM sessions WHERE token = %s", (token,))
        connection.commit()

    return jsonify({"message": "Logged out successfully"}), 200

@app.route('/bookmark', methods=['POST'])
def add_bookmark():
    data = request.json or {}
    auth_header = request.headers.get('Authorization')

    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]

    with connection.cursor() as cursor:
        cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
        user = cursor.fetchone()

        if not user:
            return jsonify({"error": "Invalid token"}), 401

        username = user['username']  # Use username instead of user_id

    recipe_id = data.get('recipe_id')
    folder_name = data.get('folder_name', '').strip()
    rating = data.get('rating', 0)

    if not recipe_id or not folder_name:
        return jsonify({"error": "Missing required fields"}), 400

    try:
        with connection.cursor() as cursor:
            # Fix: Use `username` instead of `user_id`
            cursor.execute("SELECT id FROM folders WHERE username = %s AND folder_name = %s", (username, folder_name))
            folder_row = cursor.fetchone()

            if folder_row:
                folder_id = folder_row['id']
            else:
                cursor.execute("INSERT INTO folders (username, folder_name) VALUES (%s, %s)", (username, folder_name))
                folder_id = cursor.lastrowid

            cursor.execute("INSERT IGNORE INTO folder_recipes (folder_id, recipe_id) VALUES (%s, %s)", (folder_id, recipe_id))
            cursor.execute("""
                INSERT INTO bookmarks (user_id, recipe_id, rating)
                VALUES ((SELECT user_id FROM users WHERE username = %s), %s, %s)
                ON DUPLICATE KEY UPDATE rating = VALUES(rating)
            """, (username, recipe_id, rating))

            connection.commit()

        return jsonify({"message": "Bookmark added successfully"}), 200

    except pymysql.MySQLError as e:
        return jsonify({"error": str(e)}), 500


@app.route('/bookmarks', methods=['GET'])
def get_bookmarks():
    auth_header = request.headers.get('Authorization')

    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]

    with connection.cursor() as cursor:
        cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
        user = cursor.fetchone()

        if not user:
            return jsonify({"error": "Invalid token"}), 401

        username = user['username']

    try:
        with connection.cursor() as cursor:
            # Fetch all folders for this user
            cursor.execute("SELECT id AS folder_id, folder_name FROM folders WHERE username = %s", (username,))
            folders = cursor.fetchall()

            for folder in folders:
                folder_id = folder["folder_id"]

                # Fetch recipes specific to this folder, sorted by rating (descending)
                cursor.execute("""
                    SELECT fr.recipe_id, b.rating 
                    FROM folder_recipes fr
                    LEFT JOIN bookmarks b ON fr.recipe_id = b.recipe_id 
                    AND b.user_id = (SELECT user_id FROM users WHERE username = %s)
                    WHERE fr.folder_id = %s
                    ORDER BY b.rating DESC
                """, (username, folder_id))

                recipes = cursor.fetchall()

                # Fetch `image_url` from Elasticsearch for each recipe and clean it
                for recipe in recipes:
                    es_response = es.get(index="recipes", id=str(recipe["recipe_id"]), ignore=[404])
                    if es_response and es_response.get("found"):
                        recipe["_source"] = es_response["_source"]
                        raw_image_url = recipe["_source"].get("image_url", "")
                        recipe["image_url"] = clean_text(raw_image_url)  # Apply your clean_text function

                # Ensure unique recipes in each folder
                folder["recipes"] = list({recipe["recipe_id"]: recipe for recipe in recipes}.values())

        return jsonify({"folders": folders}), 200

    except pymysql.MySQLError as e:
        return jsonify({"error": str(e)}), 500


    
@app.route('/bookmark', methods=['DELETE'])
def delete_bookmark():
    """Delete a single recipe from a folder"""
    data = request.json or {}
    auth_header = request.headers.get('Authorization')

    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]

    with connection.cursor() as cursor:
        cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
        user = cursor.fetchone()

        if not user:
            return jsonify({"error": "Invalid token"}), 401

        username = user['username']

    folder_id = data.get('folder_id')
    recipe_id = data.get('recipe_id')

    if not folder_id or not recipe_id:
        return jsonify({"error": "Missing folder_id or recipe_id"}), 400

    try:
        with connection.cursor() as cursor:
            # Delete from `folder_recipes`
            cursor.execute("DELETE FROM folder_recipes WHERE folder_id = %s AND recipe_id = %s", (folder_id, recipe_id))

            # Also delete from `bookmarks` if no other folders reference it
            cursor.execute("""
                DELETE FROM bookmarks 
                WHERE recipe_id = %s AND user_id = 
                (SELECT user_id FROM users WHERE username = %s)
                AND NOT EXISTS (
                    SELECT 1 FROM folder_recipes WHERE recipe_id = %s
                )
            """, (recipe_id, username, recipe_id))

            connection.commit()

        return jsonify({"message": "Bookmark deleted successfully"}), 200

    except pymysql.MySQLError as e:
        return jsonify({"error": str(e)}), 500


@app.route('/folder', methods=['DELETE'])
def delete_folder():
    """Delete an entire folder and all bookmarks inside it"""
    data = request.json or {}
    auth_header = request.headers.get('Authorization')

    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]

    with connection.cursor() as cursor:
        cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
        user = cursor.fetchone()

        if not user:
            return jsonify({"error": "Invalid token"}), 401

        username = user['username']

    folder_id = data.get('folder_id')

    if not folder_id:
        return jsonify({"error": "Missing folder_id"}), 400

    try:
        with connection.cursor() as cursor:
            # Delete all recipes inside the folder
            cursor.execute("DELETE FROM folder_recipes WHERE folder_id = %s", (folder_id,))

            # Remove folder itself
            cursor.execute("DELETE FROM folders WHERE id = %s AND username = %s", (folder_id, username))

            connection.commit()

        return jsonify({"message": "Folder and bookmarks deleted successfully"}), 200

    except pymysql.MySQLError as e:
        return jsonify({"error": str(e)}), 500

@app.route('/recommendations', methods=['GET'])
def recommendations():
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Invalid Authorization header format"}), 401

    token = auth_header.split(" ")[1]
    with connection.cursor() as cursor:
        cursor.execute("SELECT username FROM sessions WHERE token = %s", (token,))
        user = cursor.fetchone()
        if not user:
            return jsonify({"error": "Invalid token"}), 401
        username = user['username']

    try:
        # Only return docs that actually have an image_url field
        es_response = es.search(
            index=INDEX_NAME,
            body={
                "query": {
                    "bool": {
                        "must": [
                            {"exists": {"field": "image_url"}}
                        ]
                    }
                },
                "size": 5
            }
        )

        recommendations_list = []
        for hit in es_response.get("hits", {}).get("hits", []):
            doc = hit["_source"]
            # Clean the image_url so it’s consistently a list or string
            doc["image_url"] = clean_text(doc.get("image_url", ""))
            recommendations_list.append(doc)

        return jsonify({"recommendations": recommendations_list}), 200

    except Exception as e:
        return jsonify({"error": f"Error fetching recommendations: {str(e)}"}), 500
    

In [19]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Mar/2025 00:00:20] "OPTIONS /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:00:20] "GET /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:00:21] "GET /recommendations HTTP/1.1" 200 -
C:\Users\Ned\AppData\Local\Temp\ipykernel_39372\3524956371.py:68: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.get(index=INDEX_NAME, id=recipe_id, ignore=[404])
127.0.0.1 - - [16/Mar/2025 00:02:41] "GET /recipe/595 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:02:42] "OPTIONS /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:02:42] "GET /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:02:43] "GET /recipe/1242 HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:02:44] "GET /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2025 00:10:03] "OPTIONS /recommendations HTTP/1.1" 200 -
127.0.0.1 - - [